In [150]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep  5 09:46:49 2018
@author: Lucas de Moura Rodrigues
@ra: 14.00556-5
"""

'\nCreated on Wed Sep  5 09:46:49 2018\n@author: Lucas de Moura Rodrigues\n@ra: 14.00556-5\n'

In [151]:
import time
import numpy as np
import pandas as pd
from sklearn import linear_model,metrics
import sklearn
import matplotlib.pyplot as plt

In [152]:
tempo_inicial = time.time()

In [153]:
# le o csv
print('Le e trata os dados: ', round(time.time() - tempo_inicial, 3))
precos_casa = pd.read_csv("precos_casa_california.csv")

Le e trata os dados:  0.02


In [154]:
# ve as informacoes do Dataframe
precos_casa.describe()

,latitude,median_income
count,20445.000000,20444.000000
mean,35.643596,3.876869
std,2.129236,1.904054
min,32.540000,0.499900
25%,33.940000,2.565750
50%,34.260000,3.540300
75%,37.720000,4.756600
max,41.950000,15.000100


In [155]:
# trata o dataframe, tirando outliers
precos_casa = precos_casa.replace(' ', np.nan)
precos_casa = precos_casa.dropna()
colunas = ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
for i in colunas:
    precos_casa[i] = pd.to_numeric(precos_casa[i])
    precos_casa = precos_casa.loc[precos_casa[i] < precos_casa[i].quantile(0.9)]
precos_casa = precos_casa.reset_index(drop=True)
precos_casa.describe()

,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000
mean,35.801570,28.084291,1680.148841,345.605807,961.377167,321.913411,3.347447,154299.571443
std,2.236418,10.335483,720.276826,135.152532,395.074728,124.966131,1.243951,64728.652548
min,32.540000,1.000000,2.000000,2.000000,5.000000,2.000000,0.499900,14999.000000
25%,33.930000,19.000000,1183.000000,254.000000,691.000000,237.000000,2.366250,98500.000000
50%,34.680000,30.000000,1650.000000,345.500000,958.000000,326.000000,3.233500,152250.000000
75%,37.780000,36.000000,2161.750000,444.000000,1251.000000,417.000000,4.238600,201200.000000
max,41.950000,45.000000,4293.000000,764.000000,1827.000000,554.000000,6.278100,302100.000000


In [156]:
# separa o dataframe
dados = precos_casa.loc[:, precos_casa.columns != 'median_house_value']
target = precos_casa.loc[:, precos_casa.columns == 'median_house_value']

In [157]:
dados.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.27,37.85,40.0,751.0,184.0,409.0,166.0,1.3578,NEAR BAY
1,-122.27,37.85,42.0,1639.0,367.0,929.0,366.0,1.7135,NEAR BAY
2,-122.28,37.85,41.0,535.0,123.0,317.0,119.0,2.4038,NEAR BAY


In [158]:
target.head(3)

,median_house_value
0,147500.0
1,159800.0
2,107500.0


In [159]:
# separa o dataframe: dados_treino = 0 - 60%  dados_validacao = 60 - 100%
tamanho_precos_casa = len(precos_casa)
index_60 = int(tamanho_precos_casa * 0.6)  # primeiros 60% do dataframe
dados_treino = dados[:index_60]
dados_validacao = dados[index_60:]
target_treino = target[:index_60]
target_validacao = target[index_60:]

In [160]:
# numeriza o ocean
dic_ocean = {0: 'NEAR BAY', 1: '<1H OCEAN', 2: 'INLAND', 3: 'NEAR OCEAN', 4: 'ISLAND'}
for chave, valor in dic_ocean.items():
    dados_treino = dados_treino.replace(valor, chave)
    dados_validacao = dados_validacao.replace(valor, chave)

In [161]:
dados_treino.shape

(5744, 9)

In [162]:
dados_validacao.shape

(3830, 9)

In [163]:
# machine learning
print('Predicao dos Dados: ', round(time.time() - tempo_inicial, 3))
lm = linear_model.SGDRegressor()
lm.fit(dados_treino, target_treino)
target_predicao = lm.predict(dados_validacao)
resultado = target_validacao.reset_index(drop=True).copy()
target_predicao = [float(i) for i in target_predicao]

Predicao dos Dados:  1.087


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [164]:
sklearn.metrics.mean_squared_error(target_validacao, target_predicao)

1.6736730814201603e+30

In [165]:
# compara os dados
print('Comparando os Dados: ', round(time.time() - tempo_inicial, 3))
resultado['predicao'] = pd.Series(predicao)
resultado = resultado.rename(index=str, columns={'median_house_value': 'Valor Correto', 'predicao': 'Valor_Calculado'})
resultado['Diferenca_Absoluta'] = resultado['Valor Correto'] - resultado['Valor_Calculado']
resultado['Diferenca_Porcentagem'] = (resultado['Valor Correto'] / resultado['Valor_Calculado'] - 1) * 100

Comparando os Dados:  1.131


In [166]:
# salva o resultado em um excel
excel = pd.ExcelWriter(F"Resultado.xlsx")
precos_casa.to_excel(excel, sheet_name='Precos_Casa_Tratado')
resultado.to_excel(excel, sheet_name='Comparacao')
excel.save()

In [167]:
print('Tempo do Programa: ', round(time.time() - tempo_inicial, 3))

Tempo do Programa:  3.573
